# 1. Calculating the pressure of vapor saturation

The volatile content of a melt in equilibrium with a vapor can be used as a barometer because the sum of the partial pressures of all the vapor species must equal the total pressure (REF). This is often applied to melt inclusions to calculate magma storage depths (REF) and sub-aqueous matrix glasses to calculate eruption depths (REF).

VolFe can calculate the pressure of vapor saturation (Pvsat), melt speciation, and vapor speciation for a melt of given temperature and melt composition (including volatiles and oxygen fugacity). 

In this example we'll show you how to run this calculation for:

a) a single analysis entered as a dataframe using default options

b) a single analysis from a csv file using default options

c) all analyses in a csv file using default options

d) a set of consequative rows from a csv file using options

e) all analyses in a csv file using one set of user specified options for all analyses

f) all analyses in a csv file using different model options for each analysis

## Setting things up
First we need to import a few Python packages (including VolFe), load the species file, and set the precision of the calculations

In [1]:
# import python packages
import pandas as pd
import gmpy2 as gp
import VolFe as vf

# set calculation precision
gp.get_context().precision=50

## 1a. Calculate <i>P<sup>v</sup></i><sub>sat</sub> for a single melt composition from a dataframe using default options

We need a dataframe of the melt composition, an estimate of oxygen fugacity and Fe in the melt*, and temperature:
- Sample is just the name for this analysis
- Temperature is in 'C
- Volatile-free melt composition is in wt% oxides - all these oxides must be present in the dataframe, so set them to 0. if you have no data for them (although see not around FeOT further down this notebook)
- H2O is all hydrogen in the melt reported as H2O in wt%
- CO2ppm is all carbon in the melt reported as CO2 in ppm
- STppm is all sulfur in the melt reported as S in ppm
- Xppm is all "X" in the melt reported as X in ppm
- Fe3+FeT is the ratio of Fe3+ to FeT in the melt

*in this example we specify oxygen fugacity using Fe3+/FeT and Fe in the melt as FeOT as is quite common for melt inclusion and matrix glass analyses - other options are possible, shown later in the notebook.

The following composition is analysis TN273-01D-01-01 from REF, with a temperature chosen as 1200 'C.

In [2]:
# Define the melt composition, fO2 estimate, and T as a dictionary.
my_analysis = {'Sample':'TN273-01D-01-01',
           'T_C': 1200.,
           'SiO2': 56.98,
           'TiO2': 1.66,
           'Al2O3': 15.52,
           'FeOT': 9.47,
           'MnO': 0.24,
           'MgO': 2.96,
           'CaO': 6.49,
           'Na2O': 4.06,
           'K2O': 0.38,
           'P2O5': 0.22,
           'H2O': 1.88,
           'CO2ppm': 13.,
           'STppm': 362.83,
           'Xppm': 0.,
           'Fe3FeT': 0.171}

# Turn the dictionary into a pandas dataframe, setting the index to 0.
my_analysis = pd.DataFrame(my_analysis, index=[0])

# Show the DataFrame.
print(my_analysis)

            Sample     T_C   SiO2  TiO2  Al2O3  FeOT   MnO   MgO   CaO  Na2O  \
0  TN273-01D-01-01  1200.0  56.98  1.66  15.52  9.47  0.24  2.96  6.49  4.06   

    K2O  P2O5   H2O  CO2ppm   STppm  Xppm  Fe3FeT  
0  0.38  0.22  1.88    13.0  362.83   0.0   0.171  


Now, as we're using default options for everything, we run the calculation like so:

In [3]:
# runs the calculation
vf.P_sat_output(my_analysis)

0 TN273-01D-01-01 337.80934489141146


,sample,T_C,P_bar,SiO2_wtpc,TiO2_wtpc,Al2O3_wtpc,FeOT_wtpc,MnO_wtpc,MgO_wtpc,CaO_wtpc,...,KHOSg opt,KOSg opt,KOSg2 opt,KCOg opt,KCOHg opt,KOCSg opt,KCOs opt,carbonylsulfide opt,density opt,Date
0,TN273-01D-01-01,1200.0,337.809345,57.03956,1.661735,15.536223,9.479899,0.240251,2.963094,6.496784,...,KO97,KO97,OM22,KO97,KO97,Moussallam19,Holloway92,COS,DensityX,2024-02-15 12:59:08.761331


## 1b. Calculate <i>P<sup>v</sup></i><sub>sat</sub> for a single analysis from a csv file using default options

This time, instead of creating a dataframe in a cell for the analysis, we'll read it in from a spreadsheet. We'll use the examples_marianas csv in files and use all the default options again.

In [4]:
# Read csv to define melt composition
my_analyses = pd.read_csv("../files/example_marianas.csv") 

If there was only one analysis in the file, we'd simply run:

vf.P_sat_output(my_analysis,species)

However, there are a few so instead we tell the function to stop after the first analysis by saying last_row=1:

In [ ]:
# runs the calculation
vf.P_sat_output(my_analyses,last_row=1)

## 1c. Calculate <i>P<sup>v</sup></i><sub>sat</sub> for all analyses in a csv file using default options

Using the same example marianas file and the default options, to run all the analyses, simply don't tell it the last row you want to run. All analyses must be in consequitive rows for this to work!


In [ ]:
# runs the calculation
vf.P_sat_output(my_analyses)

## 1d. Calculate <i>P<sup>v</sup></i><sub>sat</sub> for  a set of consequative rows from a csv file using options

If we only want to run row 4 (Agr07-4) through 18 (AGR19-02-16) in that spreadsheet, we simply specify the first and last rows we want to run (remembering that row 2 in a spreadsheet - i.e., the first analysis under the headings - is actually row 0 in the dataframe).

In [ ]:
vf.P_sat_output(my_analyses,first_row=2, last_row=17)

## 1e. Calculate <i>P<sup>v</sup></i><sub>sat</sub> for all analyses in a csv file using one set of user specified options for all analyses

What if I don't want to use the default options? In that case, we create a dataframe telling VolFe what to use instead. There are lots of options that can be changed (see models csv in files for the full list, description, and options currently available). But let's say I just want to use a different solubility constant for carbon dioxide and hydrogen sulfide and treat S2 as an ideal gas. Also I don't want it to print the status during a calculation.

In [5]:
# choose the options I want - everything else will use the default options
my_models = [['carbon dioxide','Dixon97'],['hydrogen sulfide','basaltic andesite'],['y_S2','ideal'],['print status','no']]

# turn to dataframe with correct column headers and indexes    
my_models = vf.mafe_df_and_add_model_defaults(my_models)

# show what the model dataframe looks like
print(my_models)

AttributeError: module 'VolFe' has no attribute 'made_df_and_add_model_defaults'

In [ ]:
vf.P_sat_output(my_analyses,models=my_models)

One option is to load a csv with the model options listed in it (see the example models csv in the files). Or we 

## 1f. Calculate <i>P<sup>v</sup></i><sub>sat</sub> for  all analyses in a csv file using different model options for each analysis